<a href="https://colab.research.google.com/github/Satyake/Sentiment_Analysis_with-Visualisation-of-embeddings/blob/main/Sentiment_Analytis_with_reverse_word_index_with__word_trace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import tensorflow as tf
import numpy as np 
import tensorflow_datasets as tfds

In [40]:
imdb,info=tfds.load("imdb_reviews",with_info=True,as_supervised=True)

In [41]:
imdb['train']

<PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [49]:
training_sentences=[]
training_labels=[]
testing_sentences=[]
testing_labels=[]



In [50]:
train_data,test_data=imdb['train'],imdb['test']

In [51]:
for sentence,label in train_data:
  training_sentences.append(sentence.numpy().decode('utf8'))
  training_labels.append(label.numpy())
for sentence,label in test_data:
  testing_sentences.append(sentence.numpy().decode('utf8'))
  testing_labels.append(label.numpy())
training_labels=np.array(training_labels)
testing_labels=np.array(testing_labels)

In [52]:
training_labels.shape

(25000,)

In [ ]:
training_sentences

In [57]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [86]:
vocab_size=11000
embedding_dim=16
max_length=120
oov_tok='<OOV>'
tknizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tknizer.fit_on_texts(training_sentences)
sequences=tknizer.texts_to_sequences(training_sentences)
padded_sequences=pad_sequences(sequences,maxlen=max_length)
test_padded_sequences=pad_sequences(tknizer.texts_to_sequences(testing_sentences))


In [65]:
word_index=tknizer.word_index

In [82]:
reverse_word_index=dict([(value,key) for (key,value) in word_index.items()])
def decode_review(text):
  return ' '.join([reverse_word_index.get(i,'x') for i in text])


In [107]:
decode_review(padded_sequences[0])
#training_sentences[0]

"x x x this was an absolutely terrible movie don't be lured in by christopher walken or michael <OOV> both are great actors but this must simply be their worst role in history even their great acting could not redeem this movie's ridiculous storyline this movie is an early nineties us propaganda piece the most pathetic scenes were those when the <OOV> rebels were making their cases for <OOV> maria <OOV> <OOV> appeared phony and her pseudo love affair with walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning i am disappointed that there are movies like this ruining actor's like christopher <OOV> good name i could barely sit through it"

In [131]:
model=tf.keras.Sequential([
     tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
     #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1,activation='relu',return_sequences=False)),
     tf.keras.layers.GlobalAveragePooling1D(),
     tf.keras.layers.Dense(4,activation='relu'),
     tf.keras.layers.Dense(1,activation='sigmoid')
])


In [132]:
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=['accuracy'])


In [133]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 120, 16)           176000    
_________________________________________________________________
global_average_pooling1d_6 ( (None, 16)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               2176      
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 129       
Total params: 178,305
Trainable params: 178,305
Non-trainable params: 0
_________________________________________________________________


In [134]:
patience = 1
stop_patience = 3
factor = 0.5

callbacks = [
    #tf.keras.callbacks.ModelCheckpoint("classify_model.h5", save_best_only=True, verbose = 0),
   # tf.keras.callbacks.EarlyStopping(patience=stop_patience, monitor='val_accuracy', verbose=1),
    #tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=factor, patience=patience, verbose=1)
]
model.fit(padded_sequences,training_labels,epochs=10,validation_data=(test_padded_sequences,testing_labels))

Epoch 1/10
776/782 [============================>.] - ETA: 0s - loss: 0.6854 - accuracy: 0.6521WARNING:tensorflow:Model was constructed with shape (None, 120) for input KerasTensor(type_spec=TensorSpec(shape=(None, 120), dtype=tf.float32, name='embedding_8_input'), name='embedding_8_input', description="created by layer 'embedding_8_input'"), but it was called on an input with incompatible shape (None, 2332).


782/782 [==============================] - 7s 8ms/step - loss: 0.6853 - accuracy: 0.6528 - val_loss: 0.7019 - val_accuracy: 0.5000
Epoch 2/10
782/782 [==============================] - 6s 7ms/step - loss: 0.6012 - accuracy: 0.7797 - val_loss: 0.6988 - val_accuracy: 0.5004
Epoch 3/10
782/782 [==============================] - 6s 8ms/step - loss: 0.4638 - accuracy: 0.8326 - val_loss: 0.6782 - val_accuracy: 0.5073
Epoch 4/10
782/782 [==============================] - 6s 8ms/step - loss: 0.3734 - accuracy: 0.8574 - val_loss: 0.6663 - val_accuracy: 0.5144
Epoch 5/10
782/782 [==============================] - 6s 8ms/step - loss: 0.3247 - accuracy: 0.8729 - val_loss: 0.6279 - val_accuracy: 0.6669
Epoch 6/10
782/782 [==============================] - 6s 7ms/step - loss: 0.2949 - accuracy: 0.8824 - val_loss: 0.6294 - val_accuracy: 0.5727
Epoch 7/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2736 - accuracy: 0.8917 - val_loss: 0.6660 - val_accuracy: 0.5207
Epoch 8/10
782/78

In [135]:
e=model.layers[0]
weights=e.get_weights()[0]
print(weights.shape)

(11000, 16)


In [136]:
weights.shape

(11000, 16)

In [137]:
import io
out_v=io.open('vecs.tsv','w',encoding='utf8')
out_m=io.open('meta.tsv','w',encoding='utf8')
for word_num in range(1,vocab_size):
  word=reverse_word_index[word_num]
  embeddings=weights[word_num]
  out_m.write(word+'\n')
  out_v.write('\t'.join([str(x)for x in embeddings])+'\n')
out_m.close()
out_v.close()